# Federated XGBoost Tutorial
#### RISE Camp tutorial on the Federated XGBoost project, part of the MC<sup>2</sup> project.

## Single Party XGBoost on Data Subset
First we'll train an XGBoost model on a subset of the data. This simulates the federated setting in that a party will only have a subset of the data that's available to the central trusted server for training. We'll look at the performance of a XGBoost model that's only trained on this subset. 

Import the necessary libraries

In [2]:
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error

Load in and examine our training data

In [3]:
training_data_subset = pd.read_csv('/home/ubuntu/data/msd_training_data_split.csv', sep=",", header=None)
training_data_subset.head()

,0,1,2,3,4,5,6,7,8,9,...,81,82,83,84,85,86,87,88,89,90
0,2007,25.23214,-232.77465,-37.51542,-40.34335,56.11564,-55.94831,43.06882,15.46278,-38.67370,...,-15.64058,257.69408,113.59740,-90.14988,-13.41911,-72.59105,-185.49959,1.16272,-73.13128,-6.89193
1,2007,27.96974,-166.08713,-11.19265,-28.07397,-56.10902,-35.47258,23.35854,7.19973,-36.81179,...,21.49227,289.05914,-34.75972,-19.38242,2.44006,-67.78591,-46.62749,0.38383,98.98315,13.14364
2,2007,24.75152,-97.45055,-40.15226,-43.39929,-57.25665,-33.93026,-1.95605,0.93121,7.76578,...,-5.96584,573.94557,11.83355,-107.81947,-3.42495,-141.79299,-150.79400,0.55715,148.71490,-2.41587
3,2007,20.19082,-162.50028,-123.04788,-71.11772,-8.96605,-51.72176,30.53830,15.27979,-34.99486,...,-73.13628,18.76005,46.07843,-309.69087,-24.52842,-35.79334,-774.53143,3.34849,-194.68101,-41.23842
4,2007,25.10092,-189.85543,-28.69605,-34.42398,24.64007,-55.86989,63.91339,17.88235,-3.39713,...,-3.70478,40.14964,95.55738,-36.47506,-8.63102,-34.57157,-13.63610,8.25615,108.42127,3.51335


In [ ]:
y_train_subset = training_data_subset.iloc[:, 0]
y_train_subset.head()

In [ ]:
x_train_subset = training_data_subset.iloc[:, 1:]
x_train_subset.head()

Do the same with the test data

In [ ]:
test_data_subset = pd.read_csv('/data/msd_test_data_split.csv', sep=",", header=None)
y_test_subset = test_data_subset.iloc[:, 0]
x_test_subset = test_data_subset.iloc[:, 1:]

Train the model with the training data

In [ ]:
model = xgb.XGBRegressor(n_estimators=40)
model.fit(x_train_subset, y_train_subset)

Get predictions and evaluate the model with the test data

In [ ]:
preds = model.predict(x_test_subset)
np.sqrt(mean_squared_error(preds, y_test_subset))

## Single Party XGBoost on Full Dataset
Next we will look at the performance (both in terms of speed and accuracy) of training an XGBoost model on the full dataset. Training on the full dataset makes the model much more robust, and therefore helps to motivate running XGBoost in the federated setting.

In [ ]:
training_data = pd.read_csv('/data/msd_training_data.csv', sep=",", header=None)
y_train = training_data.iloc[:, 0]
x_train = training_data.iloc[:, 1:]

In [ ]:
test_data = pd.read_csv('/data/msd_test_data.csv', sep=",", header=None)
y_test = test_data.iloc[:, 0]
x_test = test_data.iloc[:, 1:]

In [ ]:
full_model = xgb.XGBRegressor(n_estimators=40)
full_model.fit(x_train, y_train)

In [ ]:
preds = full_model.predict(x_test)
np.sqrt(mean_squared_error(preds, y_test))

## Multiparty XGBoost with Distributed Data
This section will demonstrate a workflow in which each party has its own data and sends a copy of its data to the master. Therefore, all the training data is sent over the network to the master, who collects it and locally trains a model on all the data. We will also measure the number of bytes sent over the network to show the large bandwidth needed for this workflow.

In [ ]:
# If you are a worker, run this cell
!scp /data/training_data_1.csv <master_ip>:/data

If you are the master, load all the data that has been sent to your machine. For example, if three other parties sent you data, make 4 calls to `read_csv()`: one for your own data and three for the other parties' data.

In [ ]:
master_training_data = pd.read_csv('/data/msd_training_data_split.csv', sep=",", header=None)
p1_training_data = pd.read_csv('/data/msd_training_data_split.csv', sep=",", header=None)
master_training_data.shape

Concatenate all the data in preparation for training

In [ ]:
aggregated_training_data = pd.concat([master_training_data, p1_training_data])
aggregated_training_data.shape

In [ ]:
y_agg_train = aggregated_training_data.iloc[:, 0]
x_agg_train = aggregated_training_data.iloc[:, 1:]

In [ ]:
multiparty_model = xgb.XGBRegressor(n_estimators=40)
multiparty_model.fit(x_agg_train, y_agg_train)

In [ ]:
# TODO: how are we going to set up test data?
preds = multiparty_model.predict(x_test)
np.sqrt(mean_squared_error(preds, y_test))

## Federated XGBoost
We will now discuss running XGBoost in the federated setting. Unlike the previous exercise, in the federated setting all data stays on its respective machine. This eliminates the need to transfer over the network which incurs high overhead and requires significant bandwidth. Instead, in the federated setting in each iteration each party sends a summary of the update made to its model. The master then aggregates these updates, applies the aggregated update to its model, and broadcasts the new model to all parties. The parties then train locally with the new model and send the update to the master.

### Edit hosts.config 
The `hosts.config` file should contain the IPs and ports of all workers in the federation. After loading in the `hosts.config` file, modify it to contain the IPs of your new friends! Then write the new addresses back to the file by adding a magic to the top of the cell:

`%%writefile hosts.config`

Make sure to delete the `# %load hosts.config` line from the cell before saving it. We'll be continually using the `%load` and `%%writefile` magics in this tutorial to edit files.

In [ ]:
%load hosts.config

### Add SSH keys
We now need to add the master's public SSH key to all worker's `authorized_keys` file.

In [ ]:
# Run this if you're the master to get your SSH public key
!cat ~/.ssh/id_rsa.pub

In [ ]:
# Run this if you're a worker to authorize the master to SSH into your machine.
# Replace the <master_ssh_key> with the actual key
!echo "<master_ssh_key>" >> ~/.ssh/authorized_keys

### Modifying the Training/Eval Script
We will now modify the script that will be run for training and evaluation. Load it in by running the following cell. The contents of the script should appear in the cell.

In [ ]:
%load tutorial.py

### Start Job
After modifying the script, we can start our job! We use the `start_job.sh` script with the given options to do so.

The following flags must be specified when running the script.

`./start_job.sh`

* `-m | --worker-memory` string, specified as "<memory>g", e.g. 3g
    * Amount of memory on workers allocated to job
* `-p | --num-parties` integer
    * Number of parties in the federation
* `-d | --dir` string
    * Path to created subdirectory containing job script, e.g. `/home/ubuntu/mc2/federated-xgboost/risecamp`
* `-j | --job` string
    * Path to job script. This should be the parameter passed into the `--dir` option concatenated with the job script file name, e.g. `/home/ubuntu/mc2/federated-xgboost/risecamp/tutorial.py`

In [ ]:
!!./start_job.sh -p 3 -m 3g -j /home/$USER/tutorial.py